In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [4]:
evt_18_20 = pd.read_csv( loc_ts + "\\evt_18_20.csv" );
evt_21_23 = pd.read_csv( loc_ts + "\\evt_21_23.csv" );
evt_24_26 = pd.read_csv( loc_ts + "\\evt_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
ins_18_20["date"] =  pd.to_datetime( ins_18_20["date"] ); ins_18_20["_sc"] = ins_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); ins_18_20["_sc"] = ins_18_20["_sc"].dt.total_seconds();
ins_21_23["date"] =  pd.to_datetime( ins_21_23["date"] ); ins_21_23["_sc"] = ins_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); ins_21_23["_sc"] = ins_21_23["_sc"].dt.total_seconds(); 
ins_24_26["date"] =  pd.to_datetime( ins_24_26["date"] ); ins_24_26["_sc"] = ins_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); ins_24_26["_sc"] = ins_24_26["_sc"].dt.total_seconds(); 

In [8]:
ins_18_20_first_ins = ins_18_20.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_21_23_first_ins = ins_21_23.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )
ins_24_26_first_ins = ins_24_26.sort_values( by = ["ref_hash","_sc"], ascending = True ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_ins"

In [10]:
rh_18_20 = ins_18_20_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = ins_21_23_first_ins[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = ins_18_20_first_ins[ ["_sc"] ]
tg_21_23 = ins_21_23_first_ins[ ["_sc"] ]

## <span style="color:yellow"> ============================================================================================================================ </span> 

## <span style="color:green"> **Horario del primer evento en ese ventana** </span> 

Al agregarle la transformacion senoidal para agregarle perdiodicidad a la hora, la prediccion del algoritmo fue peor por lo que decidimos no agregarla

In [16]:
hr_f_evt_18_20 = rh_18_20.copy()
hr_f_evt_21_23 = rh_21_23.copy()

first_evt_hour_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_evt_hour_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_evt_hour_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_evt_hour_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_evt_hour_18_20["hour_frt_evt"] = pd.to_datetime( first_evt_hour_18_20["date"] ).dt.hour
first_evt_hour_21_23["hour_frt_evt"] = pd.to_datetime( first_evt_hour_21_23["date"] ).dt.hour

first_evt_hour_18_20.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )
first_evt_hour_21_23.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )

hr_f_evt_18_20 = hr_f_evt_18_20.merge( first_evt_hour_18_20, how = "left", on = "ref_hash" )
hr_f_evt_21_23 = hr_f_evt_21_23.merge( first_evt_hour_21_23, how = "left", on = "ref_hash" )

#hr_f_evt_18_20["hour_frt_evt"] = hr_f_evt_18_20["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
#hr_f_evt_21_23["hour_frt_evt"] = hr_f_evt_21_23["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

hr_f_evt_18_20.to_csv( loc_ftr + "\\hr_f_evt_18_20.csv", index = False )
hr_f_evt_21_23.to_csv( loc_ftr + "\\hr_f_evt_21_23.csv", index = False )

Si bien hay varios valores NaN en el feature, lo decidimos probar de igual manera, ya que los algoritmos de boosting aceptan valores NaNs.

* https://datascience.stackexchange.com/questions/15305/how-does-xgboost-learn-what-are-the-inputs-for-missing-values/15306#15306
* The procedure is described in [their paper, section 3.4: Sparsity aware split-finding](https://arxiv.org/pdf/1603.02754v3.pdf).


## <span style="color:red"> **Cantidad de Eventos por dispositivo en la ventana previa a la conversion** </span>

In [33]:
cant_evt_18_20 = rh_18_20
cant_evt_21_23 = rh_21_23

cant_evt_18_20 = cant_evt_18_20.merge( evt_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_21_23 = cant_evt_21_23.merge( evt_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_evt_18_20.fillna( 0, inplace = True )
cant_evt_21_23.fillna( 0, inplace = True )

cant_evt_18_20.to_csv( loc_ftr + "\\cant_evt_18_20.csv", index = False )
cant_evt_21_23.to_csv( loc_ftr + "\\cant_evt_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de eventos atribuidas por dispositivo** </span> 

In [86]:
cevt_atr_18_20 = rh_18_20.copy()
cevt_atr_21_23 = rh_21_23.copy()

cant_atr_evt_18_20 = evt_18_20[["ref_hash", "attributed"]]; cant_atr_evt_18_20["attributed"] = cant_atr_evt_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_21_23 = evt_21_23[["ref_hash", "attributed"]]; cant_atr_evt_21_23["attributed"] = cant_atr_evt_21_23["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_18_20 = cant_atr_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_21_23 = cant_atr_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_18_20 = cevt_atr_18_20.merge( cant_atr_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_21_23 = cevt_atr_21_23.merge( cant_atr_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_18_20.to_csv( loc_ftr + "\\cevt_atr_18_20.csv", index = False )
cevt_atr_21_23.to_csv( loc_ftr + "\\cevt_atr_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer evento en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer evento

In [46]:
frst_evt_18_20 = rh_18_20.copy()
frst_evt_21_23 = rh_21_23.copy()

first_event_rh_ins_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_event_rh_ins_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_event_rh_ins_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_event_rh_ins_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_event_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_event_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

frst_evt_18_20 = frst_evt_18_20.merge( first_event_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_evt_21_23 = frst_evt_21_23.merge( first_event_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_evt_18_20.to_csv( loc_ftr + "\\frst_evt_18_20.csv", index = False )
frst_evt_21_23.to_csv( loc_ftr + "\\frst_evt_21_23.csv", index = False )

## <span style="color:green"> **Tipo de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese tipo sobre el total de eventos realizados)

In [152]:
kind_evt_18_20 = rh_18_20.copy()
kind_evt_21_23 = rh_21_23.copy()

main_kind_evt_18_20 = evt_18_20.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()
main_kind_evt_21_23 = evt_21_23.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()

main_kind_evt_18_20 = main_kind_evt_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_18_20["cant_evt"] )
main_kind_evt_21_23 = main_kind_evt_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_21_23["cant_evt"] ) 

kind_evt_18_20 = kind_evt_18_20.merge( main_kind_evt_18_20, how = "left", on = "ref_hash" )
kind_evt_21_23 = kind_evt_21_23.merge( main_kind_evt_21_23, how = "left", on = "ref_hash" )

cant_kind_evt_18_20 = evt_18_20[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )
cant_kind_evt_21_23 = evt_21_23[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )

kind_evt_18_20 = kind_evt_18_20.merge( cant_kind_evt_18_20, how = "left", on = "kind" ).drop( "kind", axis = 1 )
kind_evt_21_23 = kind_evt_21_23.merge( cant_kind_evt_21_23, how = "left", on = "kind" ).drop( "kind", axis = 1 )

kind_evt_18_20["kind_mean"] = kind_evt_18_20["cant_kind"] / len(evt_18_20); kind_evt_18_20.fillna( kind_evt_18_20["kind_mean"].isnull().sum() / len(evt_18_20), inplace = True ); del(kind_evt_18_20["cant_kind"])
kind_evt_21_23["kind_mean"] = kind_evt_21_23["cant_kind"] / len(evt_21_23); kind_evt_21_23.fillna( kind_evt_21_23["kind_mean"].isnull().sum() / len(evt_21_23), inplace = True ); del(kind_evt_21_23["cant_kind"])

kind_evt_18_20.to_csv( loc_ftr + "\\kind_evt_18_20.csv", index = False )
kind_evt_21_23.to_csv( loc_ftr + "\\kind_evt_21_23.csv", index = False )

## <span style="color:red"> **Cantidad de eventos atribuidas por dispositivo** </span> 

In [ ]:
cevt_atr_18_20 = rh_18_20.copy()
cevt_atr_21_23 = rh_21_23.copy()

cant_atr_evt_18_20 = evt_18_20[["ref_hash", "attributed"]]; cant_atr_evt_18_20["attributed"] = cant_atr_evt_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_21_23 = evt_21_23[["ref_hash", "attributed"]]; cant_atr_evt_21_23["attributed"] = cant_atr_evt_21_23["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_18_20 = cant_atr_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_21_23 = cant_atr_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_18_20 = cevt_atr_18_20.merge( cant_atr_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_21_23 = cevt_atr_21_23.merge( cant_atr_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_18_20.to_csv( loc_ftr + "\\cevt_atr_18_20.csv", index = False )
cevt_atr_21_23.to_csv( loc_ftr + "\\cevt_atr_21_23.csv", index = False )

## <span style="color:red"> **Aplicaciones que mas realizaron eventos cada usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [ ]:
mapp_evt_18_20 = rh_18_20.copy()
mapp_evt_21_23 = rh_21_23.copy()

app_mas_evt_18_20 = evt_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_21_23 = evt_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_18_20 = app_mas_evt_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_18_20["cant_evt"] )
app_mas_evt_21_23 = app_mas_evt_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_21_23["cant_evt"] ) 

mapp_evt_18_20 = mapp_evt_18_20.merge( app_mas_evt_18_20, how = "left", on = "ref_hash" )
mapp_evt_21_23 = mapp_evt_21_23.merge( app_mas_evt_21_23, how = "left", on = "ref_hash" )

mapp_evt_18_20["to_count"] = 1; mapp_evt_18_20["application_id"] = mapp_evt_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_18_20); del(mapp_evt_18_20["to_count"])
mapp_evt_21_23["to_count"] = 1; mapp_evt_21_23["application_id"] = mapp_evt_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_21_23); del(mapp_evt_21_23["to_count"])

#Completamos los NaNs con el promedio de su cantidad
mapp_evt_18_20["application_id"].fillna( mapp_evt_18_20["application_id"].isnull().sum() / len( mapp_evt_18_20 ) , inplace = True )
mapp_evt_21_23["application_id"].fillna( mapp_evt_21_23["application_id"].isnull().sum() / len( mapp_evt_21_23 ), inplace = True )

mapp_evt_18_20.to_csv( loc_ftr + "\\mapp_evt_18_20.csv", index = False )
mapp_evt_21_23.to_csv( loc_ftr + "\\mapp_evt_21_23.csv", index = False )

## <span style="color:red"> **Cantidad de eventos realizados con WiFi** </span>  

In [ ]:
wifi_evt_18_20 = rh_18_20.copy()
wifi_evt_21_23 = rh_21_23.copy()

cant_wifi_evt_18_20 = evt_18_20[["ref_hash", "wifi"]]; cant_wifi_evt_18_20["wifi"] = cant_wifi_evt_18_20["wifi"].apply( lambda x: 1 if x else 0 );
cant_wifi_evt_21_23 = evt_21_23[["ref_hash", "wifi"]]; cant_wifi_evt_21_23["wifi"] = cant_wifi_evt_21_23["wifi"].apply( lambda x: 1 if x else 0 );

cant_wifi_evt_18_20 = cant_wifi_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_wifi_evt_21_23 = cant_wifi_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

wifi_evt_18_20 = wifi_evt_18_20.merge( cant_wifi_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
wifi_evt_21_23 = wifi_evt_21_23.merge( cant_wifi_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

wifi_evt_18_20.to_csv( loc_ftr + "\\wifi_evt_18_20.csv", index = False )
wifi_evt_21_23.to_csv( loc_ftr + "\\wifi_evt_21_23.csv", index = False )

## <span style="color:red"> **Tiempo hasta la ultimo evento en esa ventana** </span> 

In [12]:
last_evt_18_20 = rh_18_20.copy()
last_evt_21_23 = rh_21_23.copy()

last_event_rh_ins_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_event_rh_ins_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_event_rh_ins_18_20["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_event_rh_ins_21_23["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_event_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
last_event_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

last_evt_18_20 = last_evt_18_20.merge( last_event_rh_ins_18_20, how = "left", on = "ref_hash" )
last_evt_21_23 = last_evt_21_23.merge( last_event_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_evt_18_20.to_csv( loc_ftr + "\\last_evt_18_20.csv", index = False )
last_evt_21_23.to_csv( loc_ftr + "\\last_evt_21_23.csv", index = False )

## <span style="color:green"> **Hizo eventos entre 21 hs y 3 hs (Noche)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [61]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_21_3'] = (evt_18_20['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)
evt_21_23['evt_21_3'] = (evt_21_23['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)

evt_night_18_20 = rh_18_20.copy()
evt_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_18_20 = evt_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_night_21_23 = evt_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_night_18_20["evt_21_3"] =  (evt_night_18_20["evt_21_3"] > 1).astype('int8')
evt_night_21_23["evt_21_3"] =  (evt_night_21_23["evt_21_3"] > 1).astype('int8')

evt_night_18_20['me_evt_21_3'] = ( evt_night_18_20.groupby('evt_21_3').transform('count') ) / len(evt_night_18_20['evt_21_3'])
evt_night_21_23['me_evt_21_3'] = ( evt_night_21_23.groupby('evt_21_3').transform('count') ) / len(evt_night_21_23['evt_21_3'])

del evt_night_18_20['evt_21_3']
del evt_night_21_23['evt_21_3']

evt_night_18_20.to_csv( loc_ftr + "\\evt_ngme_18_20.csv", index = False )
evt_night_21_23.to_csv( loc_ftr + "\\evt_ngme_21_23.csv", index = False )

del evt_18_20['evt_21_3']
del evt_21_23['evt_21_3']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [62]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_21_3'] = (evt_18_20['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)
evt_21_23['evt_21_3'] = (evt_21_23['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)

evt_night_18_20 = rh_18_20.copy()
evt_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_18_20 = evt_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_night_21_23 = evt_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_night_18_20["evt_21_3"] =  (evt_night_18_20["evt_21_3"] > 1).astype('int8')
evt_night_21_23["evt_21_3"] =  (evt_night_21_23["evt_21_3"] > 1).astype('int8')

evt_night_18_20.to_csv( loc_ftr + "\\evt_ngoh_18_20.csv", index = False )
evt_night_21_23.to_csv( loc_ftr + "\\evt_ngoh_21_23.csv", index = False )

del evt_18_20['evt_21_3']
del evt_21_23['evt_21_3']

## <span style="color:green"> **Hizo eventos entre 4 hs y 10 hs (Mañana)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [63]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_4_10'] = (evt_18_20['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)
evt_21_23['evt_4_10'] = (evt_21_23['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)

evt_morn_18_20 = rh_18_20.copy()
evt_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_18_20 = evt_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_morn_21_23 = evt_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_morn_18_20["evt_4_10"] =  (evt_morn_18_20["evt_4_10"] > 1).astype('int8')
evt_morn_21_23["evt_4_10"] =  (evt_morn_21_23["evt_4_10"] > 1).astype('int8')

evt_morn_18_20['me_evt_4_10'] = ( evt_morn_18_20.groupby('evt_4_10').transform('count') ) / len(evt_morn_18_20['evt_4_10'])
evt_morn_21_23['me_evt_4_10'] = ( evt_morn_21_23.groupby('evt_4_10').transform('count') ) / len(evt_morn_21_23['evt_4_10'])

del evt_morn_18_20['evt_4_10']
del evt_morn_21_23['evt_4_10']

evt_morn_18_20.to_csv( loc_ftr + "\\evt_mrme_18_20.csv", index = False )
evt_morn_21_23.to_csv( loc_ftr + "\\evt_mrme_21_23.csv", index = False )

del evt_18_20['evt_4_10']
del evt_21_23['evt_4_10']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [64]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_4_10'] = (evt_18_20['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)
evt_21_23['evt_4_10'] = (evt_21_23['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)

evt_morn_18_20 = rh_18_20.copy()
evt_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_18_20 = evt_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_morn_21_23 = evt_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_morn_18_20["evt_4_10"] =  (evt_morn_18_20["evt_4_10"] > 1).astype('int8')
evt_morn_21_23["evt_4_10"] =  (evt_morn_21_23["evt_4_10"] > 1).astype('int8')

evt_morn_18_20.to_csv( loc_ftr + "\\evt_mroh_18_20.csv", index = False )
evt_morn_21_23.to_csv( loc_ftr + "\\evt_mroh_21_23.csv", index = False )

del evt_18_20['evt_4_10']
del evt_21_23['evt_4_10']

## <span style="color:green"> **Hizo eventos entre 11 hs y 15 hs (Mediodia)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [65]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_11_15'] = (evt_18_20['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)
evt_21_23['evt_11_15'] = (evt_21_23['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)

evt_midday_18_20 = rh_18_20.copy()
evt_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_18_20 = evt_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_midday_21_23 = evt_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_midday_18_20["evt_11_15"] =  (evt_midday_18_20["evt_11_15"] > 1).astype('int8')
evt_midday_21_23["evt_11_15"] =  (evt_midday_21_23["evt_11_15"] > 1).astype('int8')

evt_midday_18_20['me_evt_11_15'] = ( evt_midday_18_20.groupby('evt_11_15').transform('count') ) / len(evt_midday_18_20['evt_11_15'])
evt_midday_21_23['me_evt_11_15'] = ( evt_midday_21_23.groupby('evt_11_15').transform('count') ) / len(evt_midday_21_23['evt_11_15'])

del evt_midday_18_20['evt_11_15']
del evt_midday_21_23['evt_11_15']

evt_midday_18_20.to_csv( loc_ftr + "\\me_evt_mdme_18_20.csv", index = False )
evt_midday_21_23.to_csv( loc_ftr + "\\me_evt_mdme_21_23.csv", index = False )

del evt_18_20['evt_11_15']
del evt_21_23['evt_11_15']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [66]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_11_15'] = (evt_18_20['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)
evt_21_23['evt_11_15'] = (evt_21_23['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)

evt_midday_18_20 = rh_18_20.copy()
evt_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_18_20 = evt_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_midday_21_23 = evt_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_midday_18_20["evt_11_15"] =  (evt_midday_18_20["evt_11_15"] > 1).astype('int8')
evt_midday_21_23["evt_11_15"] =  (evt_midday_21_23["evt_11_15"] > 1).astype('int8')

evt_midday_18_20.to_csv( loc_ftr + "\\evt_mdoh_18_20.csv", index = False )
evt_midday_21_23.to_csv( loc_ftr + "\\evt_mdoh_21_23.csv", index = False )

del evt_18_20['evt_11_15']
del evt_21_23['evt_11_15']

## <span style="color:green"> **Hizo eventos entre 16 hs y 20 hs (Tarde)** </span>

#### <span style="color:Orange"> **Mean Encoding** </span> 

In [67]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_16_20'] = (evt_18_20['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)
evt_21_23['evt_16_20'] = (evt_21_23['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)

evt_after_18_20 = rh_18_20.copy()
evt_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_18_20 = evt_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_after_21_23 = evt_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_after_18_20["evt_16_20"] =  (evt_after_18_20["evt_16_20"] > 1).astype('int8')
evt_after_21_23["evt_16_20"] =  (evt_after_21_23["evt_16_20"] > 1).astype('int8')

evt_after_18_20['me_evt_16_20'] = ( evt_after_18_20.groupby('evt_16_20').transform('count') ) / len(evt_after_18_20['evt_16_20'])
evt_after_21_23['me_evt_16_20'] = ( evt_after_21_23.groupby('evt_16_20').transform('count') ) / len(evt_after_21_23['evt_16_20'])

del evt_after_18_20['evt_16_20']
del evt_after_21_23['evt_16_20']

evt_after_18_20.to_csv( loc_ftr + "\\evt_afme_18_20.csv", index = False )
evt_after_21_23.to_csv( loc_ftr + "\\evt_afme_21_23.csv", index = False )

del evt_18_20['evt_16_20']
del evt_21_23['evt_16_20']

#### <span style="color:Orange"> **One-Hot Encoding** </span> 

In [68]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_16_20'] = (evt_18_20['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)
evt_21_23['evt_16_20'] = (evt_21_23['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)

evt_after_18_20 = rh_18_20.copy()
evt_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_18_20 = evt_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_after_21_23 = evt_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_after_18_20["evt_16_20"] =  (evt_after_18_20["evt_16_20"] > 1).astype('int8')
evt_after_21_23["evt_16_20"] =  (evt_after_21_23["evt_16_20"] > 1).astype('int8')

evt_after_18_20.to_csv( loc_ftr + "\\evt_afoh_18_20.csv", index = False )
evt_after_21_23.to_csv( loc_ftr + "\\evt_afoh_21_23.csv", index = False )

del evt_18_20['evt_16_20']
del evt_21_23['evt_16_20']